In [1]:
# -*- coding: utf-8 -*-
"""
Script FINAL (Versão 4.4 - Filtragem de Sinopse por Tamanho): Análise de Frequência de Palavras em Sinopses de Filmes de Terror (1980-1989).

Este script coleta 200 sinopses de filmes do gênero terror, lançados entre 1980 e 1989,
focando apenas nos critérios de ano e gênero, sem considerar popularidade.
Implementa uma filtragem heurística de sinopses muito longas (provavelmente reviews ou long plots)
para tentar obter apenas o resumo do enredo.
Utiliza a biblioteca Cinemagoer (IMDbPY) para a coleta e SpaCy para pré-processamento.
A saída durante a coleta é concisa, e o resultado final exibe as 100 palavras mais frequentes.
"""

import imdb
import spacy
from collections import Counter
import pandas as pd
import time
import random

# --- Carregar o modelo SpaCy de inglês ---
try:
    nlp = spacy.load("en_core_web_sm")
    print("Modelo SpaCy 'en_core_web_sm' carregado com sucesso.")
except OSError:
    print("\nERRO CRÍTICO: Modelo SpaCy 'en_core_web_sm' não encontrado ou não carregado.")
    print("Por favor, certifique-se de que você executou o comando:")
    print("  python -m spacy download en_core_web_sm")
    print("no seu Anaconda Prompt/Terminal (com o ambiente 'terror_nlp' ativo) ANTES de executar este script.")
    exit()

# --- Configurações Finais do Projeto ---
ia = imdb.IMDb()
ANOS_INICIO = 1980
ANOS_FIM = 1989
GENERO_DESEJADO = 'horror'
QUANTIDADE_FILMES = 200
TOP_TERMOS_SINOPSE = 100

IMDB_BASE_URL = "https://www.imdb.com/title/tt"

# --- Novo Filtro para Sinopses ---
MAX_WORDS_IN_PLOT = 500 # Limite heurístico de palavras para considerar uma sinopse válida
                        # Sinopses normais raramente excedem 500 palavras. Avaliações podem ter milhares.

print(f"Iniciando a coleta de {QUANTIDADE_FILMES} filmes do gênero '{GENERO_DESEJADO}' lançados entre {ANOS_INICIO} e {ANOS_FIM} (filtrando sinopses muito longas)...")

filmes_para_processar = []
filmes_ids_coletados = set()

print("\n--- Coleta de Filmes: Aguarde, este processo pode levar alguns minutos... ---")
for year in range(ANOS_INICIO, ANOS_FIM + 1):
    if len(filmes_para_processar) >= QUANTIDADE_FILMES:
        break

    try:
        search_results = ia.search_movie(f'horror {year}') 
        
        for movie in search_results:
            if len(filmes_para_processar) >= QUANTIDADE_FILMES:
                break
            
            try:
                ia.update(movie, ['plot', 'main'])
                time.sleep(0.1 + (random.random() * 0.1))

                if movie.get('year') == year:
                    genres = [g.lower() for g in movie.get('genres', [])]
                    if GENERO_DESEJADO in genres:
                        plot_summary_raw = movie.get('plot', movie.get('plots', ['N/A']))
                        if isinstance(plot_summary_raw, list):
                            plot_summary_temp = " ".join(plot_summary_raw)
                        else:
                            plot_summary_temp = plot_summary_raw
                        
                        # --- Aplicação do Filtro de Tamanho ---
                        if plot_summary_temp and plot_summary_temp.strip() != '':
                            # Tokeniza temporariamente para contar palavras
                            doc_temp = nlp(plot_summary_temp)
                            num_words = len([token for token in doc_temp if token.is_alpha]) # Conta apenas palavras alfabéticas
                            
                            if num_words > MAX_WORDS_IN_PLOT:
                                # print(f"  Sinopse muito longa para '{movie.get('title')}' ({num_words} palavras). Descartando ou truncando.")
                                # Podemos descartar ou tentar pegar só um trecho inicial
                                # Por enquanto, vamos descartar para ter sinopses mais limpas
                                plot_summary = None # Descartar se for muito longa
                            else:
                                plot_summary = plot_summary_temp # Aceitar se estiver dentro do limite
                        else:
                            plot_summary = None # Sinopse vazia ou inválida
                        # --- Fim do Filtro de Tamanho ---

                        if plot_summary and movie.movieID not in filmes_ids_coletados:
                            filmes_para_processar.append({
                                'title': movie.get('title'),
                                'year': movie.get('year'),
                                'genres': movie.get('genres'),
                                'plot': plot_summary,
                                'imdb_link': f"{IMDB_BASE_URL}{movie.movieID}"
                            })
                            filmes_ids_coletados.add(movie.movieID)
                            print(f"Coletado ({len(filmes_para_processar)}/{QUANTIDADE_FILMES}): {movie.get('title')} ({movie.get('year')}) - {IMDB_BASE_URL}{movie.movieID}")
            except Exception as e:
                pass
    except Exception as e:
        pass

print(f"\n--- Coleta de Filmes Concluída: {len(filmes_para_processar)} de {QUANTIDADE_FILMES} filmes coletados. ---")

if not filmes_para_processar:
    print("Nenhum filme foi encontrado com os critérios especificados. Verifique as configurações ou tente novamente.")
else:
    textos_sinopses = [f['plot'] for f in filmes_para_processar if f['plot'] is not None] # Filtra sinopses None
    texto_completo_sinopses = " ".join(textos_sinopses)

    print("\n--- Análise das Sinopses ---")
    print("Iniciando pré-processamento das sinopses com SpaCy...")
    doc_sinopses = nlp(texto_completo_sinopses.lower())

    palavras_sinopses_filtradas = [
        token.text for token in doc_sinopses
        if not token.is_stop and not token.is_punct and token.is_alpha and len(token.text) > 1
    ]

    print("Calculando frequência das palavras nas sinopses...")
    frequencia_palavras_sinopses = Counter(palavras_sinopses_filtradas)
    top_termos_sinopses = frequencia_palavras_sinopses.most_common(TOP_TERMOS_SINOPSE)

    print(f"\n--- {TOP_TERMOS_SINOPSE} Palavras Mais Frequentes nas Sinopses ---")
    for palavra, frequencia in top_termos_sinopses:
        print(f"'{palavra}': {frequencia}")

print("\nProcesso concluído!")

Modelo SpaCy 'en_core_web_sm' carregado com sucesso.
Iniciando a coleta de 200 filmes do gênero 'horror' lançados entre 1980 e 1989 (filtrando sinopses muito longas)...

--- Coleta de Filmes: Aguarde, este processo pode levar alguns minutos... ---
Coletado (1/200): Hammer House of Horror (1980) - https://www.imdb.com/title/tt0080231
Coletado (2/200): Inferno (1980) - https://www.imdb.com/title/tt0080923
Coletado (3/200): Chamber 13 - Tales of Horror (1980) - https://www.imdb.com/title/tt9426590
Coletado (4/200): Without Warning (1980) - https://www.imdb.com/title/tt0081764
Coletado (5/200): Alligator (1980) - https://www.imdb.com/title/tt0080354
Coletado (6/200): The Godsend (1980) - https://www.imdb.com/title/tt0080802
Coletado (7/200): Blood Beach (1980) - https://www.imdb.com/title/tt0082083
Coletado (8/200): White Cannibal Queen (1980) - https://www.imdb.com/title/tt0078936
Coletado (9/200): Macabre (1980) - https://www.imdb.com/title/tt0080764
Coletado (10/200): Monstroid (1980) -

In [ ]:
# --- Célula Adicional: Verificação de Ocorrência de Termos Específicos ---

# Esta célula deve ser executada APÓS o script principal, na mesma sessão do Jupyter.
# As variáveis 'filmes_para_processar' e 'nlp' já devem estar definidas.

print("\n--- Verificação de Ocorrência de Termos Específicos em Sinopses ---")

# Lista de termos que você quer verificar (em minúsculas)
termos_para_verificar = ['horror'] # Adicione ou remova termos aqui

for i, filme in enumerate(filmes_para_processar):
    plot = filme.get('plot', '')
    title = filme.get('title', 'Título Desconhecido')
    imdb_link = filme.get('imdb_link', 'Link Desconhecido')

    # Processa a sinopse individualmente, igual ao processo geral
    doc_individual = nlp(plot.lower())
    
    # Extrai tokens (palavras) filtradas da sinopse individual
    palavras_na_sinopse = [
        token.text for token in doc_individual
        if not token.is_stop and not token.is_punct and token.is_alpha and len(token.text) > 1
    ]

    encontrados_no_filme = []
    
    # Verifica a presença de cada termo na sinopse processada
    for termo in termos_para_verificar:
        if termo in palavras_na_sinopse:
            count = palavras_na_sinopse.count(termo)
            encontrados_no_filme.append(f"'{termo}': {count} vez(es)")
    
    if encontrados_no_filme:
        print(f"\nFilme {i+1}: '{title}' ({filme.get('year')})")
        print(f"  Link IMDb: {imdb_link}")
        print(f"  Termos encontrados: {', '.join(encontrados_no_filme)}")
        print(f"  Trecho da Sinopse (para contexto):")
        # Mostra um trecho da sinopse original para dar contexto
        # Encontra a primeira ocorrência de um dos termos e mostra o texto ao redor
        plot_lower = plot.lower()
        context_found = False
        for termo in termos_para_verificar:
            if termo in plot_lower:
                start_index = max(0, plot_lower.find(termo) - 50) # 50 caracteres antes
                end_index = min(len(plot_lower), plot_lower.find(termo) + len(termo) + 50) # 50 caracteres depois
                context = plot[start_index:end_index]
                print(f"    ...{context}...")
                context_found = True
                break # Mostra contexto para o primeiro termo encontrado
        if not context_found: # Se nenhum termo foi encontrado, mas a lista 'encontrados_no_filme' não está vazia (algum erro lógico)
            print("    (Contexto não exibido, erro de busca de termo para contexto)")

if not filmes_para_processar:
    print("\nNenhum filme coletado no script principal para verificar.")

print("\nVerificação de termos concluída!")